## Similarity Measures

Run similarity metrics for 10 companies with the same report date and group into quintiles.

In [1]:
#import text file of 10 companies from 2017-06-30 report date and 2016-06-30 report date
import pandas as pd
rdd = pd.read_table('manualscrapequintile.txt', sep = "|") #import text file with pipe delimiter
df = pd.DataFrame(rdd) #convert to pandas df

In [2]:
list(df) #list Dataframe columns

['cik ', ' ticker ', ' date ', ' year ', ' quarter ', ' text']

In [3]:
df_sort = df.sort_values([' ticker ', ' year ']) #sort by ticker and year

In [4]:
df_sort['text_list'] = df_sort.groupby((' ticker ', ' year '))[' text'].apply(lambda x: list(x)).tolist() #create list from text

In [5]:
df_sort.reset_index(drop = 'index') #reset index after sort

,cik,ticker,date,year,quarter,text,text_list
0,1018724,AMZN,2016-07-29,2016,2,Please carefully consider the following risk ...,[ Please carefully consider the following risk...
1,1018724,AMZN,2017-08-01,2017,2,Please carefully consider the following risk ...,[ Please carefully consider the following risk...
2,858470,COG,2016-07-29,2016,2,For additional information about the risk fac...,[ For additional information about the risk fa...
3,858470,COG,2017-07-28,2017,2,For additional information about the risk fac...,[ For additional information about the risk fa...
4,27940,DAL,2016-07-13,2016,2,“Item 1A. Risk Factors” of our Form 10-K incl...,[ “Item 1A. Risk Factors” of our Form 10-K inc...
5,27940,DAL,2017-07-13,2017,2,"""Item 1A. Risk Factors"" of our Form 10-K incl...","[ ""Item 1A. Risk Factors"" of our Form 10-K inc..."
6,1015780,ETFC,2016-08-04,2016,2,There have been no material changes in the Co...,[ There have been no material changes in the C...
7,1015780,ETFC,2017-08-03,2017,2,There have been no material changes in the Co...,[ There have been no material changes in the C...
8,511143,IBM,2016-07-26,2016,2,Except for the historical information and dis...,[ Except for the historical information and di...
9,511143,IBM,2017-07-25,2017,2,Except for the historical information and dis...,[ Except for the historical information and di...


### Parsing and Simularity Functions

Code block that parsing the list of words and sets up similarity functions.

In [53]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

# Cosine similarity function
def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

# Jaccard Similarity Function
def get_jaccard(vec1, vec2):
    return float(len(vec1.intersection(vec2))*1.0/len(vec1.union(vec2)))

def get_simple(vec1, vec2):
    return float(len(vec1.intersection(vec2))*1.0 / len(vec1))

# Word Vector Format needed for cosine similarity (ie words with counts)
def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

# Word Vector Format needed for jaccard similarity (vector of words)
def text_to_vector_js(text):
     words = WORD.findall(text)
     return words

def text_to_vector_simple(text):
     words = WORD.findall(text)
     return words

# function to calculate cosine similarity when two quarters are passed
def calc_cosine(quarterCurrent, quarterOld):
    text1 = quarterCurrent
    text2 = quarterOld
    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)
    cosine = get_cosine(vector1, vector2)
    return cosine

# function to calculate jaccard similarity when two quarters are passed
def calc_jaccard(quarterCurrent, quarterOld):
    textjs1 = quarterCurrent
    textjs2 = quarterOld
    vectorjs1 = set(text_to_vector_js(textjs1))
    vectorjs2 = set(text_to_vector_js(textjs2))
    jaccard = get_jaccard(vectorjs1, vectorjs2)
    return jaccard
                 
# function to calculate simple similarity when two quarters are passed
def calc_simple(quarterCurrent, quarterOld):
    textsimple1 = quarterCurrent
    textsimple2 = quarterOld
    vectorsimple1 = set(text_to_vector_simple(textsimple1))
    vectorsimple2 = set(text_to_vector_simple(textsimple2))
    simple = get_simple(vectorsimple1, vectorsimple2)
    return simple

In [54]:
#empty list for similarity metrics
cos_sim = []
jac_sim = []
simple_sim = []
duplicate = []

#loop through each row and calculate similarity measures if the tickers are the same
#assign a duplicate if the two quarters have already been analyzed so we can exclude later
for i in range(len(df_sort)-1):
    if df_sort.iloc[i][' ticker '] == df_sort.iloc[i +1][' ticker ']:
        cos_value = calc_cosine(df_sort.iloc[i]['text_list'][0], df_sort.iloc[i+1]['text_list'][0])
        cos_sim.append(cos_value)
        j_value = calc_jaccard(df_sort.iloc[i]['text_list'][0], df_sort.iloc[i+1]['text_list'][0])
        jac_sim.append(j_value)
        simple_value = calc_simple(df_sort.iloc[i]['text_list'][0], df_sort.iloc[i+1]['text_list'][0])
        simple_sim.append(simple_value)
        duplicate.append(0)
    else:
        cos_sim.append(0)
        jac_sim.append(0)
        simple_sim.append(0)
        duplicate.append(1)

# append a 0 for the last row in the file, which by default is a duplicate
cos_sim.append(0)
jac_sim.append(0)
simple_sim.append(0)
duplicate.append(1)

In [55]:
# append similarity lists as a column in dataframe
df_sort['cosine_similarity'] = cos_sim
df_sort['jaccard_similarity'] = jac_sim
df_sort['simple_similarity'] = simple_sim
df_sort['duplicate'] = duplicate

In [56]:
df_sort

,cik,ticker,date,year,quarter,text,text_list,cosine_similarity,jaccard_similarity,simple_similarity,duplicate
7,1018724,AMZN,2016-07-29,2016,2,Please carefully consider the following risk ...,[ Please carefully consider the following risk...,0.999429,0.964165,0.992345,0
6,1018724,AMZN,2017-08-01,2017,2,Please carefully consider the following risk ...,[ Please carefully consider the following risk...,0.000000,0.000000,0.000000,1
9,858470,COG,2016-07-29,2016,2,For additional information about the risk fac...,[ For additional information about the risk fa...,0.971429,0.933333,0.965517,0
8,858470,COG,2017-07-28,2017,2,For additional information about the risk fac...,[ For additional information about the risk fa...,0.000000,0.000000,0.000000,1
11,27940,DAL,2016-07-13,2016,2,“Item 1A. Risk Factors” of our Form 10-K incl...,[ “Item 1A. Risk Factors” of our Form 10-K inc...,0.598046,0.205357,0.207207,0
10,27940,DAL,2017-07-13,2017,2,"""Item 1A. Risk Factors"" of our Form 10-K incl...","[ ""Item 1A. Risk Factors"" of our Form 10-K inc...",0.000000,0.000000,0.000000,1
13,1015780,ETFC,2016-08-04,2016,2,There have been no material changes in the Co...,[ There have been no material changes in the C...,0.970588,0.931034,0.964286,0
12,1015780,ETFC,2017-08-03,2017,2,There have been no material changes in the Co...,[ There have been no material changes in the C...,0.000000,0.000000,0.000000,1
3,511143,IBM,2016-07-26,2016,2,Except for the historical information and dis...,[ Except for the historical information and di...,1.000000,1.000000,1.000000,0
2,511143,IBM,2017-07-25,2017,2,Except for the historical information and dis...,[ Except for the historical information and di...,0.000000,0.000000,0.000000,1


In [57]:
df_final = df_sort[df_sort['duplicate'] != 1] #remove any rows which are the duplicate for each ticker

In [61]:
import numpy as np

# cosine similarity quintile values
# quantile set to 0.20 for quintile
cos_quintile1 = df_final['cosine_similarity'].quantile(0.2)
cos_quintile2 = df_final['cosine_similarity'].quantile(0.4)
cos_quintile3 = df_final['cosine_similarity'].quantile(0.6)
cos_quintile4 = df_final['cosine_similarity'].quantile(0.8)

# function for determining quintile
def cos_quintile_rank (row):
    if row['cosine_similarity'] <= cos_quintile1:
        return 1
    if row['cosine_similarity'] <= cos_quintile2:
        return 2
    if row['cosine_similarity'] <= cos_quintile3:
        return 3
    if row['cosine_similarity'] <= cos_quintile4:
        return 4
    if row['cosine_similarity'] > cos_quintile4:
        return 5

# apply quintile based on each rows value
df_final['cosine_quintile'] = df_final.apply(lambda row: cos_quintile_rank(row), axis = 1)

# jacaard similarity quintile values
# quantile set to 0.20 for quintile
jac_quintile1 = df_final['jaccard_similarity'].quantile(0.2)
jac_quintile2 = df_final['jaccard_similarity'].quantile(0.4)
jac_quintile3 = df_final['jaccard_similarity'].quantile(0.6)
jac_quintile4 = df_final['jaccard_similarity'].quantile(0.8)

# function for determining quintile
def jac_quintile_rank (row):
    if row['jaccard_similarity'] <= jac_quintile1:
        return 1
    if row['jaccard_similarity'] <= jac_quintile2:
        return 2
    if row['jaccard_similarity'] <= jac_quintile3:
        return 3
    if row['jaccard_similarity'] <= jac_quintile4:
        return 4
    if row['jaccard_similarity'] > jac_quintile4:
        return 5

# apply quintile based on each rows value
df_final['jaccard_quintile'] = df_final.apply(lambda row: jac_quintile_rank(row), axis = 1)

# simple similarity quintile values
# quantile set to 0.20 for quintile
simple_quintile1 = df_final['simple_similarity'].quantile(0.2)
simple_quintile2 = df_final['simple_similarity'].quantile(0.4)
simple_quintile3 = df_final['simple_similarity'].quantile(0.6)
simple_quintile4 = df_final['simple_similarity'].quantile(0.8)

# function for determining quintile
def simple_quintile_rank (row):
    if row['simple_similarity'] <= simple_quintile1:
        return 1
    if row['simple_similarity'] <= simple_quintile2:
        return 2
    if row['simple_similarity'] <= simple_quintile3:
        return 3
    if row['simple_similarity'] <= simple_quintile4:
        return 4
    if row['simple_similarity'] > simple_quintile4:
        return 5

# apply quintile based on each rows value
df_final['simple_quintile'] = df_final.apply(lambda row: simple_quintile_rank(row), axis = 1)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

In [62]:
list(df_final)

['cik ',
 ' ticker ',
 ' date ',
 ' year ',
 ' quarter ',
 ' text',
 'text_list',
 'cosine_similarity',
 'jaccard_similarity',
 'simple_similarity',
 'duplicate',
 'cosine_quintile',
 'jaccard_quintile',
 'simple_quintile']

In [63]:
df_final.drop(['duplicate'], axis = 1) #final DF, dropping duplicate column which is no longer needed

,cik,ticker,date,year,quarter,text,text_list,cosine_similarity,jaccard_similarity,simple_similarity,cosine_quintile,jaccard_quintile,simple_quintile
7,1018724,AMZN,2016-07-29,2016,2,Please carefully consider the following risk ...,[ Please carefully consider the following risk...,0.999429,0.964165,0.992345,5,4,5
9,858470,COG,2016-07-29,2016,2,For additional information about the risk fac...,[ For additional information about the risk fa...,0.971429,0.933333,0.965517,3,4,4
11,27940,DAL,2016-07-13,2016,2,“Item 1A. Risk Factors” of our Form 10-K incl...,[ “Item 1A. Risk Factors” of our Form 10-K inc...,0.598046,0.205357,0.207207,1,1,1
13,1015780,ETFC,2016-08-04,2016,2,There have been no material changes in the Co...,[ There have been no material changes in the C...,0.970588,0.931034,0.964286,3,3,3
3,511143,IBM,2016-07-26,2016,2,Except for the historical information and dis...,[ Except for the historical information and di...,1.000000,1.000000,1.000000,5,5,5
1,50863,INTC,2016-08-01,2016,2,"The risks described in Part I, Item 1A, ""Risk...","[ The risks described in Part I, Item 1A, ""Ris...",0.909603,0.647541,0.647541,2,2,1
15,200406,JNJ,2016-08-03,2016,2,This Form 10-Q contains forward-looking state...,[ This Form 10-Q contains forward-looking stat...,0.862641,0.377391,0.828244,1,1,2
17,55785,KMB,2016-07-25,2016,2,Certain matters contained in this report conc...,[ Certain matters contained in this report con...,0.986929,0.931937,0.951872,4,3,3
19,1141391,MA,2016-07-28,2016,2,For a discussion of the Company’s risk factor...,[ For a discussion of the Company’s risk facto...,0.914659,0.870968,0.900000,2,2,2
5,66740,MMM,2016-08-01,2016,2,Provided below is a cautionary discussion of ...,[ Provided below is a cautionary discussion of...,0.998937,0.965798,0.965798,4,5,4


In [65]:
df_final.to_csv('/Users/z013nx1/Documents/stocksWithSimilarityMetricsQuintiles.txt') # export to text